In [2]:
import pandas as pd
import requests
from pathlib import Path
import subprocess
import os
from google.cloud import storage
from google.cloud.storage import transfer_manager
import re
import pickle
import numpy as np

In [3]:
df = pd.read_csv("data/pifsc_products_detections_annotations.csv")

# Fixes bug with hawaii 
df['hawaii_num'] = df[df['flac_compressed_xwav_object'].str.contains("pipan_hawaii_0/")]['flac_compressed_xwav_object'].str.split("/", expand=True).iloc[:, -1].str.split("_", expand=True).loc[:, 2].astype(float)

df.loc[:, "flac_compressed_xwav_object"] = df.apply(lambda x: x['flac_compressed_xwav_object'].replace("pipan_hawaii_0/", f"pipan_hawaii_{int(x['hawaii_num'])}/") if not np.isnan(x["hawaii_num"]) else x["flac_compressed_xwav_object"], axis=1)

df.head()

,audit_name,flac_compressed_xwav_object,subchunk_index,label_is_strong,implicit_negatives,label,begin_rel_subchunk,end_rel_subchunk,begin_utc,end_utc,hawaii_num
0,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,1,True,True,Other,0.213,0.213,2005-06-06 13:00:00.213000+00:00,2005-06-06 13:00:00.213000+00:00,NaN
1,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,13,True,True,Other,0.108,0.108,2005-06-06 14:15:00.108000+00:00,2005-06-06 14:15:00.108000+00:00,NaN
2,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Other,0.143,0.143,2005-07-02 04:00:00.143000+00:00,2005-07-02 04:00:00.143000+00:00,NaN
3,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,17,True,True,Other,0.213,0.213,2005-07-02 05:15:00.213000+00:00,2005-07-02 05:15:00.213000+00:00,NaN
4,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,22,True,True,Other,0.213,0.213,2005-08-01 10:15:00.213000+00:00,2005-08-01 10:15:00.213000+00:00,NaN


In [13]:
files = list(df['flac_compressed_xwav_object'].unique())
files = [re.sub("gs://noaa-passive-bioacoustic/", "", f) for f in files]
# files = files[0:1000]

def download_many_blobs_with_transfer_manager(bucket_name, blob_names, destination_directory="", workers=8):
    storage_client = storage.Client.create_anonymous_client()
    
    bucket = storage_client.bucket(bucket_name)
    
    results = transfer_manager.download_many_to_path(bucket, blob_names, destination_directory=destination_directory, max_workers=workers)
    
    bad_results = []
    for name, result in zip(blob_names, results):
        if isinstance(result, Exception):
            bad_results.append((name, result))
            print(f"Failed to download {name} due to exception {result}")
        else:
            print(f"Download of {name} succeeded")

    return bad_results
files[:5], len(files)

(['pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050606_123845.d20.x.flac',
  'pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050702_031345.d20.x.flac',
  'pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050801_074615.d20.x.flac',
  'pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050801_110500.d20.x.flac',
  'pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050908_025000.d20.x.flac'],
 5294)

In [14]:
downloaded_files = []
for dirpath, dirnames, filenames in os.walk("D:/SonarData/downloads"):
    for filename in filenames:
        downloaded_files.append(filename)
        
to_download = [f for f in files if f.split("/")[-1] not in downloaded_files]
to_download[:5], len(to_download)

(['pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120419_235000.df20.x.flac',
  'pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120420_221115.df20.x.flac',
  'pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120421_203230.df20.x.flac',
  'pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120423_233229.df20.x.flac',
  'pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120426_183615.df20.x.flac'],
 4682)

5082

In [16]:
bad_results = download_many_blobs_with_transfer_manager("noaa-passive-bioacoustic", 
                                                        to_download[:400],
                                                        destination_directory="D:/SonarData/downloads",
                                                        workers=8)

Download of pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120419_235000.df20.x.flac succeeded
Download of pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120420_221115.df20.x.flac succeeded
Download of pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120421_203230.df20.x.flac succeeded
Download of pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120423_233229.df20.x.flac succeeded
Download of pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120426_183615.df20.x.flac succeeded
Download of pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120427_165730.df20.x.flac succeeded
Download of pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120428_151845.df20.x.flac succeeded
Download of pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120429_134000.df20.x.flac succeeded
Download of pifsc/audio/pipan/equator/pipan_equator_01/audio/Equator_A_01_120501_164000.df20.x.flac succeeded
Download o

In [17]:
len(bad_results)

0

In [11]:
with open("bad_results.pkl", 'wb') as f:
    pickle.dump(bad_results, f)

In [78]:
with open("bad_results.pkl", 'rb') as f:
    bad_results = pickle.load(f)
bad_files = [f[0] for f in bad_results]
bad_file_names = {f.split("/")[-1] for f in bad_files}
list(bad_file_names)[:5]

['Hawaii_K_16_140207_060115.df20.x.flac',
 'Hawaii_K_16_140105_233000.df20.x.flac',
 'Hawaii_K_11_110924_072800.df20.x.flac',
 'Hawaii_K_14_121222_090345.df20.x.flac',
 'Hawaii_K_14_121228_162345.df20.x.flac']

In [79]:
for dirpath, dirnames, filenames in os.walk("D:/SonarData/downloads"):
    for filename in filenames:
        if filename in bad_file_names:
            os.remove(os.path.join(dirpath, filename))

In [16]:
# Uncompress data
files = os.listdir("D:/SonarData/flat")
new_files = [i for i in files if i.endswith(".flac")]
for i, file in enumerate(new_files):
    if i % 50 == 0:
        print(f"Processing file {i}")
    subprocess.run(f"C:/Windows/flac.exe -df --delete-input-file --preserve-modtime --keep-foreign-metadata D:/SonarData/flat/{file}", shell=True, capture_output=True)

Processing file 0
Processing file 50
Processing file 100
Processing file 150
Processing file 200
Processing file 250
Processing file 300
Processing file 350
Processing file 400
Processing file 450
Processing file 500
Processing file 550
Processing file 600
Processing file 650
Processing file 700
Processing file 750
Processing file 800
Processing file 850
Processing file 900
Processing file 950


In [12]:
new_files

['Cross_A_01_050514_014345.d20.x.flac',
 'Cross_A_01_050518_040230.d20.x.flac',
 'Cross_A_01_050520_183000.d20.x.flac',
 'Cross_A_01_050520_212731.d20.x.flac',
 'Cross_A_01_050521_004500.d20.x.flac',
 'Cross_A_01_050527_003345.d20.x.flac',
 'Cross_A_01_050606_123845.d20.x.flac',
 'Cross_A_01_050612_151845.d20.x.flac',
 'Cross_A_01_050613_232350.d20.x.flac',
 'Cross_A_01_050630_004730.d20.x.flac',
 'Cross_A_01_050630_222845.d20.x.flac',
 'Cross_A_01_050701_112115.d20.x.flac',
 'Cross_A_01_050702_031345.d20.x.flac',
 'Cross_A_01_050703_170730.d20.x.flac',
 'Cross_A_01_050717_151615.d20.x.flac',
 'Cross_A_01_050722_115500.d20.x.flac',
 'Cross_A_01_050801_074615.d20.x.flac',
 'Cross_A_01_050801_110500.d20.x.flac',
 'Cross_A_01_050908_025000.d20.x.flac',
 'Cross_A_02_051120_000000.d20.x.flac']

In [8]:
len(files)

1012

In [15]:
files = os.listdir("D:/SonarData/flat")
len([i for i in files if i.endswith(".flac")])

992